<a href="https://colab.research.google.com/github/AndreyLFR/Library_np_pd/blob/master/Course_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проект - прогнозирование дефолта корпоративного заемщика. Данные использованы из открытых источников (СПАРК). Событие дефолт - иск от кредитной организации.
Данные собраны самостоятельно

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import ascii_letters
from sklearn.model_selection import train_test_split

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

1. Загрузка датасета

In [131]:
df_target = pd.read_excel('/content/drive/MyDrive/Course_work/Targets.xlsx')
df_target.drop_duplicates(subset=['ИНН'], inplace=True)

df_spark = pd.read_excel('/content/drive/MyDrive/Course_work/ML_study_SPARK.xlsx')
df_spark.drop_duplicates(subset=['Код налогоплательщика'], inplace=True)

df = df_target.merge(df_spark, how='left', left_on='ИНН', right_on='Код налогоплательщика')

2. Разобьем на датасет на тестовый и для обучения

In [132]:
y = df['Дефолт'].values
X = df.drop('Дефолт', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

3. Создадим класс обработки датасета

Переменными не влияющими на дефолт являются ИНН, Ответчик, Отчетность, Код налогоплательщика, Наименование, Регистрационный номер.

Также в столбце Сумма незавершенных исков в роли ответчика, тыс. RUB отражены неверные данные, так как они не дату за год до дефолта, а на дату выгрузки данных.

EBIT - важный показатель. Отражает маржинальность бизнес, способность обслуживать финансовый долг. Заменить его медианой будет некорректно. У половины значений с target 1 нет данных по EBIT. Следует дозапросить этот показатель. На текущий момент атрибут исключаю

Вид деятельности. В зависимости от отрасли меняются финансовые показатели. В рамках примера применена разбивка на 5 групп отраслей

ОС - основные средства. Значение не может быть меньше 0. Аналогичный подход как к ОС применю к остальным финансовым показателям кроме 'Капитал АППГ', 'Капитал'. Сначала уберу отрицательные значения, где с экономической точки зрения их не может быть

На абсолютных финансовых показателях некорректно работать с выбросами. После расчета относительных показателей проведу работу с выбросами

Регион регистрации влияет на дефолтность, но требует дополнительного погружения в социально-экономические аспекты регионов для их сегментации

In [133]:
class DataPipeline:

    def __init__(self):
      self.extra_variables = ['Ответчик', 'Отчетность', 'Отчетный год', 'Код налогоплательщика', 'Наименование',
                   'Регистрационный номер', 'Сумма незавершенных исков в роли ответчика, тыс. RUB', 'ИНН']
      self.columns_FP = ['ОС', 'ДФЛ АППГ', 'ДФЛ', 'ВНА', 'КФЛ АППГ', 'КФЛ', 'Активы', 'Капитал АППГ', 'Капитал', 'ДКиЗ', 'ККиЗ', 'Совокупный долг, АППГ',
                    'Совокупный долг', 'Выручка, АППГ', 'Выручка', 'Проценты к уплате']
      self.columns_Nan_Median = ['Обор КЗ АППГ', 'Обор КЗ', 'Обор запасы АППГ', 'Обор запасы', 'Обор ДЗ АППГ', 'Обор ДЗ']
      self.col_only_positive = ['ДФЛ АППГ', 'ДФЛ', 'ВНА', 'КФЛ АППГ', 'КФЛ', 'Активы', 'ДКиЗ', 'ККиЗ', 'Совокупный долг, АППГ',
                    'Совокупный долг', 'Выручка, АППГ', 'Выручка', 'Проценты к уплате', 'ОС']
      self.columns_OP = ['Обор КЗ АППГ', 'Обор КЗ', 'Обор запасы АППГ', 'Обор запасы', 'Обор ДЗ АППГ', 'Обор ДЗ',
                         'Коэффициент соотношения заемных и собственных средств,']
      self.list_dop_cols = ['Динамика выручки', 'Долг к выручке', 'Долг к выручке к АППГ', 'Динамика долга', 'Капитал к долгу', 'Аналит капитал к долгу',
             'Капитал к ВБ', 'Аналит капитал к ВБ', 'Динамика обор запасов АППГ', 'Динамика обор ДЗ АППГ', 'Динамика обор КЗ АППГ', 'ОС к ВБ', 'Динамика капитала', 'Динамика аналит капитала']
      self.regions = ['Санкт-Петербург', 'Москва', 'Хакасия (Республика)', 'Кировская область', 'Пензенская область', 'Магаданская область', 'Республика Татарстан', 'Пермский край', 'Красноярский край', 'Московская область', 'Владимирская область', 'Нижегородская область', 'Курганская область', 'Сахалинская область', 'Карелия (Республика)', 'Бурятия (Республика)', 'Приморский край', 'Северная Осетия-Алания (Республика)', 'Краснодарский край', 'Ярославская область', 'Рязанская область', 'Волгоградская область', 'Башкортостан (Республика)', 'Ленинградская область', 'Свердловская область', 'Белгородская область', 'Ставропольский край', 'Тюменская область', 'Омская область', 'Тамбовская область', 'Челябинская область', 'Новосибирская область', 'Кабардино-Балкарская Республика', 'Томская область', 'Новгородская область', 'Дагестан (Республика)', 'Алтайский край', 'Иркутская область', 'Севастополь', 'Удмуртская Республика', 'Саратовская область', 'Саха (Республика) (Якутия)', 'Республика Крым', 'Курская область', 'Калужская область', 'Самарская область', 'Ростовская область', 'Оренбургская область', 'Тульская область', 'Липецкая область', 'Ивановская область', 'Смоленская область', 'Хабаровский край', 'Алтай (Республика)', 'Амурская область', 'Чувашская Республика-Чувашия', 'Чеченская Республика', 'Забайкальский край', 'Камчатский край', 'Ульяновская область', 'Калининградская область', 'Псковская область', 'Костромская область', 'Чукотский автономный округ', 'Воронежская область', 'Кемеровская область', 'Мордовия (Республика)', 'Марий Эл (Республика)', 'Архангельская область', 'Вологодская область', 'Тверская область', 'Астраханская область', 'Брянская область', 'Коми (Республика)', 'Еврейская автономная область', 'Мурманская область', 'Орловская область', 'Карачаево-Черкесская Республика', 'Калмыкия (Республика)', 'Адыгея (Республика) (Адыгея)', 'Тыва (Республика)', 'Ингушетия (Республика)', 'Байконур']


    def fit(self, X_train, y_train):
      clf = RandomForestClassifier(max_depth=9, max_features=8, n_estimators=250)
      clf.fit(X_train, y_train)
      joblib.dump(clf, "/content/drive/MyDrive/Course_work/model.pkl")
      return clf

    def predict(self, X_test):
      clf = joblib.load("/content/drive/MyDrive/Course_work/model.pkl")
      return clf.predict(X_test)

    def internal_func_owner(self, list_own):
      try:
        count = len(list_own)
      except:
        count = 0
      return count

    def internal_func_foreigner(self, owners):
      if owners:
        for owner in owners:
          list_bool_char = list(map(lambda c: c in ascii_letters, owner))
          if True in list_bool_char:
            return 1
            break
        return 0

    def internal_func_SSH(self, str_):
      if '-' in str_:
        result = int(str_.split('-')[-1].replace(' ', ''))
      elif '>' in str_:
        result = int(str_.split('>')[-1].replace(' ', ''))
      else:
        result = int(str_.replace(' ', ''))
      return result

    def internal_func_INDUSTRY(self, df):
        ref_col = 'Вид деятельности/отрасль'
        for i in df.index:
            if 'озничная' in df.at[i, ref_col] and 'орговля' in df.at[i, ref_col]:
                df.loc[i, ref_col] = 'Розничная торговля'
            elif 'птовая' in df.at[i, ref_col] and 'орговля' in df.at[i, ref_col]:
                df.loc[i, ref_col] = 'Оптовая торговля'
            elif 'астениеводство' in df.at[i, ref_col] or 'ивотноводство' in df.at[i, ref_col] or 'ыращивание' in df.at[i, ref_col]:
                df.loc[i, ref_col] = 'Сельское хозяйство'
            elif 'роизводство' in df.at[i, ref_col]:
                df.loc[i, ref_col] = 'Производство'
            else:
                df.loc[i, ref_col] = 'Иная'
        return df

    def gen_new_attr(self, df):
      df['СОК'] = df['Капитал'] + df['ДКиЗ'] - df['ВНА'] - df['КФЛ']
      df['Аналитический СК'] = df['Капитал'] - df['ДФЛ'] - df['КФЛ']
      df['Аналитический СК АППГ'] = df['Капитал АППГ'] - df['ДФЛ АППГ'] - df['КФЛ АППГ']
      df['Динамика выручки'] = df['Выручка'] / df['Выручка, АППГ']
      df['Долг к выручке'] = df['Совокупный долг'] / df['Выручка']
      df['Долг к выручке к АППГ'] = df['Совокупный долг, АППГ'] / df['Выручка, АППГ']
      df['Динамика долга'] = df['Долг к выручке'] / df['Долг к выручке к АППГ']
      df['Пол или отр СОК'] = [1 if element >= 0 else 0 for element in df['СОК']]
      df['Капитал к долгу'] = df['Капитал'] / df['Совокупный долг']
      df['Аналит капитал к долгу'] = df['Аналитический СК'] / df['Совокупный долг']
      df['Капитал к ВБ'] = df['Капитал'] / df['Активы']
      df['Аналит капитал к ВБ'] = df['Аналитический СК'] / df['Активы']
      df['Динамика обор запасов АППГ'] = df['Обор запасы'] / df['Обор запасы АППГ']
      df['Динамика обор ДЗ АППГ'] = df['Обор ДЗ'] / df['Обор ДЗ АППГ']
      df['Динамика обор КЗ АППГ'] = df['Обор КЗ'] / df['Обор КЗ АППГ']
      df['ОС к ВБ'] = df['ОС'] / df['Активы']
      df['Динамика капитала'] = df['Капитал'] / df['Капитал АППГ']
      df['Динамика аналит капитала'] = df['Аналитический СК'] / df['Аналитический СК АППГ']
      return df

    def rebalancing(self, X_train, y_train):
        train = pd.DataFrame(data=X_train)
        train['Дефолт'] = y_train
        train_1 = train[train['Дефолт'] == 1]
        train_0 = train[train['Дефолт'] == 0]
        train_0 = train_0.sample(train_1.shape[0] * 2, random_state=0)
        train_1 = pd.concat([train_1, train_1.copy()])
        train_bal = pd.concat([train_1, train_0])
        df = pd.DataFrame(data=train_bal)
        y = df['Дефолт'].values
        X = df.drop('Дефолт', axis=1)
        return X, y

    def transform(self, X):
      #исключение лишних атрибутов
      X.drop(self.extra_variables, axis=1, inplace=True)
      #обработка Nan атрибута Сайт в сети интернет
      X['Сайт в сети Интернет'] = X['Сайт в сети Интернет'].fillna(0)
      X['Сайт'] = np.where(X['Сайт в сети Интернет']!=0, 1, 0)
      X.drop('Сайт в сети Интернет', axis=1, inplace=True)
      X.drop('EBIT', axis=1, inplace=True)
      #обработка Nan атрибута Совладельцы, Приоритетный
      X['Совладельцы, Приоритетный'] = X['Совладельцы, Приоритетный'].fillna(0)
      X['Совладельцы'] = np.where(X['Совладельцы, Приоритетный']!=0, X['Совладельцы, Приоритетный'].str.split('\n'), 0)
      #новый атрибут - количество владельцев
      X['Количество владельцев'] = np.where(X['Совладельцы']!=0, X['Совладельцы'].apply(lambda x: self.internal_func_owner(list_own=x)), 0)
      #новый атрибут - собственник иностранец
      X['С иностранным участием'] = np.where(X['Совладельцы']!=0, X['Совладельцы'].apply(lambda x: self.internal_func_foreigner(x)), 0)
      X.drop(['Совладельцы', 'Совладельцы, Приоритетный'], axis=1, inplace=True)
      #обработка Nan ССЧ
      X['ССЧ'] = X['ССЧ'].fillna('999999')
      X['ССЧ'] = X['ССЧ'].apply(lambda x: self.internal_func_SSH(x))
      X.loc[X['ССЧ'] == 999999, 'ССЧ'] = X['ССЧ'].loc[(X['ССЧ'] != 999999)].median()
      #Nan меняю на 0, исходя из экономического смысла
      for col in self.columns_FP:
        X[col] = X[col].fillna(0)
      #Nan меняю на median, исходя из экономического смысла
      for col in self.columns_Nan_Median:
        X[col] = X[col].fillna(999999)
        X.loc[(X[col] == 999999)|(X[col] < 0), col] = X[col].loc[(X[col] != 999999)&(X[col] > 0)].median()
      X['Коэффициент соотношения заемных и собственных средств,'] = X['Коэффициент соотношения заемных и собственных средств,'].fillna(999999)
      X.loc[X['Коэффициент соотношения заемных и собственных средств,'] == 999999, 'Коэффициент соотношения заемных и собственных средств,'] = X['Коэффициент соотношения заемных и собственных средств,'].loc[X['Коэффициент соотношения заемных и собственных средств,'] != 999999].median()
      #Работа с выбросами
      X['ССЧ'] = np.where(X['ССЧ']>X['ССЧ'].quantile(.975), X['ССЧ'].quantile(.975), X['ССЧ'])
      X['Возраст'] = np.where(X['Возраст компании, лет']>X['Возраст компании, лет'].quantile(.975), X['Возраст компании, лет'].quantile(.975), X['Возраст компании, лет'])
      X.drop('Возраст компании, лет', axis=1, inplace=True)
      X = self.internal_func_INDUSTRY(X)
      for col in self.col_only_positive:
        X[col] = np.where(X[col]<0, 0, X[col])
      for col in self.columns_OP:
        X[col] = np.where(X[col]>X[col].quantile(.975), X[col].quantile(.975), X[col])
      #Добавление показателей - финансовые коэффициенты
      X = self.gen_new_attr(X)
      #Обработка вновь созданных атрибутов
      for col in self.list_dop_cols:
        m = X[X[col].notna()][col].median()
        X[col] = X[col].fillna(m)
        X[col] = np.where(X[col]>X[col].quantile(.95), X[col].quantile(.95), X[col])
        X[col] = np.where(X[col]<X[col].quantile(.01), X[col].quantile(.01), X[col])
      #Обрабатываю номинальные переменные
      X = pd.get_dummies(data=X, drop_first=True, columns=['Вид деятельности/отрасль'])
      X.drop('Регион регистрации', axis=1, inplace=True)
      return X

In [134]:
dataset = DataPipeline()
X_train_transform = dataset.transform(X=X_train)
X_train_bal, y_train_bal = dataset.rebalancing(X_train=X_train_transform, y_train=y_train)
clf = dataset.fit(X_train=X_train_bal, y_train=y_train_bal)

X_test_transform = dataset.transform(X=X_test)
y_pred = dataset.predict(X_test = X_test_transform)
accuracy_score(y_test, y_pred)

0.840669014084507

In [135]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

def results(y_test, y_pred, time=0):
    report = classification_report(y_test, y_pred, target_names=['0', '1'])
    print(report)
    print('\nПлощадь под ROC-кривой - ' + str(round(roc_auc_score(y_test, y_pred), 4)))

In [136]:
results(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.98      0.84      0.91     18836
           1       0.31      0.80      0.44      1612

    accuracy                           0.84     20448
   macro avg       0.64      0.82      0.67     20448
weighted avg       0.93      0.84      0.87     20448


Площадь под ROC-кривой - 0.8233


In [137]:
r2_score(y_test, y_pred)

-1.194058471002415

Точность модели на уровне 84% приемлемая. При этом для моделей классификации дефолтов большее значение имеет показатель recall. Ориентироваться на r2 в моделях классификации неверно

Модель правильно выявила 80% дефолтов.

In [138]:
#from sklearn.model_selection import GridSearchCV
#import numpy as np

#parameters = {
#    'n_estimators': [150, 200, 250],
#    'max_features': np.arange(5, 9),
#    'max_depth': np.arange(5, 10),
#}

#clf = GridSearchCV(
#    estimator=RandomForestClassifier(),
#    param_grid=parameters,
#    scoring='accuracy',
#    cv=5,
#)

#clf.fit(X_train_bal, y_train_bal)
#clf.best_params_
#accuracy_score(y_valid, y_pred)